영화 포스터 이미지 분석에 필요한 데이터 전처리 과정은 매우 중요하며, 모델의 성능에 큰 영향을 미칠 수 있습니다. 각 단계별로 자세히 설명드리겠습니다.

### 1. **이미지 크기 조정 (Resizing)**
   - CNN과 같은 딥러닝 모델은 입력 이미지 크기가 고정되어 있어야 합니다. 따라서 모든 포스터 이미지를 동일한 크기로 조정해야 합니다. 보통 224x224, 256x256 등의 표준 크기로 조정하며, 모델의 성능과 처리 속도에 맞게 크기를 선택합니다.
   - 이미지 크기 조정 시 원본 비율을 유지하거나, 필요에 따라 비율을 왜곡할 수도 있지만, 왜곡으로 인해 이미지 특징이 손실되지 않도록 주의해야 합니다.

### 2. **데이터 증강 (Data Augmentation)**
   - 데이터 증강은 훈련 데이터의 다양성을 높이기 위한 기법입니다. 이미지의 개수가 충분하지 않거나, 과적합(overfitting)을 방지하려는 경우 사용됩니다. 다음과 같은 방법을 사용할 수 있습니다:
     - **회전 (Rotation)**: 이미지를 다양한 각도로 회전합니다.
     - **수평/수직 반전 (Flipping)**: 이미지를 좌우 또는 상하로 뒤집습니다.
     - **크기 조정 및 자르기 (Zooming and Cropping)**: 이미지를 확대하거나 특정 부분만 잘라서 모델이 다양한 부분을 학습하도록 합니다.
     - **밝기 및 색상 변화 (Brightness and Color Shifts)**: 이미지의 밝기, 대비 또는 색상을 랜덤하게 변경하여 이미지 변동성에 대한 모델의 내성을 높입니다.
   - 데이터 증강은 학습 중에 실시간으로 적용할 수 있습니다.

### 3. **이미지 정규화 (Normalization)**
   - 이미지 데이터를 학습하기 전에, 각 픽셀 값을 일정한 범위로 정규화해야 합니다. 일반적으로 각 픽셀의 값은 0~255 사이에 있으므로, 이를 0~1 사이로 스케일링하거나 평균을 0으로 하고 표준편차를 1로 만드는 방식(Z-score)을 사용할 수 있습니다.
   - 이는 모델이 학습하는 데 있어 안정성을 높이고, 학습 속도를 빠르게 하며, 수렴 과정을 돕습니다.

### 4. **이미지 레이블링 (Label Encoding)**
   - 포스터 이미지가 분류할 장르에 맞게 레이블이 필요합니다. 다중 장르의 경우, 장르당 다중 레이블로 표현하거나, 원핫 인코딩(one-hot encoding) 방식을 사용할 수 있습니다. 예를 들어, 액션, 드라마, 코미디 등의 장르를 각각 인코딩하여 학습에 사용합니다.

### 5. **이미지 데이터 분할 (Train/Test Split)**
   - 학습, 검증, 테스트 데이터를 분할해야 합니다. 일반적으로 70:20:10 또는 80:10:10 비율로 데이터를 나누어 훈련 데이터, 검증 데이터, 테스트 데이터를 구성합니다. 이를 통해 모델의 성능을 객관적으로 평가할 수 있습니다.

### 6. **흑백 또는 컬러 전환**
   - RGB 이미지(컬러)로 학습할지, Grayscale(흑백) 이미지로 학습할지 결정합니다. 컬러 이미지가 더 많은 정보를 포함하지만, 데이터 크기 및 처리 속도를 고려할 때 흑백 이미지를 사용할 수도 있습니다.
   
### 7. **이미지 압축 및 파일 형식 변환**
   - 이미지의 크기나 형식(JPEG, PNG 등)이 다양할 수 있으므로, 모델의 처리 효율성을 높이기 위해 동일한 파일 형식으로 변환하고 적절히 압축합니다. 압축 시 이미지의 품질이 저하되지 않도록 주의해야 합니다.

### 8. **중복 제거 및 이상치 처리**
   - 데이터셋에서 중복된 이미지나 장르와 관계없는 노이즈가 포함된 이미지를 제거합니다. 품질이 매우 낮거나 분류할 수 없는 이미지는 제거하는 것이 좋습니다.

이 과정들을 철저하게 수행하면 영화 포스터 이미지를 분석하는 데 필요한 데이터를 적절히 준비할 수 있습니다. 데이터 전처리 과정이 잘 이루어지면, 모델이 이미지를 더 잘 학습하고 성능이 향상될 가능성이 큽니다.

In [2]:
# image resizing
# 포스터마다 가로 세로 비율이 다르다
# 가로 세로 비율을 유지하며 크기를 조절하고 겉에 패딩을 추가한다

from PIL import Image
import os

def resize_with_padding(image_path, target_size):
    img = Image.open(image_path)
    # img.thumbnail(target_size, Image.ANTIALIAS)  # 비율 유지 리사이즈
    # AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'
    img.thumbnail(target_size, Image.LANCZOS)  # 비율 유지 리사이즈

    # 배경색 지정 (예: 흰색)
    new_img = Image.new("RGB", target_size, (0,0,0))
    
    # 위치 계산
    img_x = (target_size[0] - img.size[0]) // 2
    img_y = (target_size[1] - img.size[1]) // 2
    new_img.paste(img, (img_x, img_y))
    
    return new_img

input_dir = r'G:\내 드라이브\project_poster\movie_posters\poster_0.jpg'
output_dir = r'C:\ex\project_poster\preprocessed'
# new_size = (224, 224)
new_size = (500,500)

os.makedirs(output_dir, exist_ok=True)

resized_img = resize_with_padding(input_dir, new_size)
resized_img.save(os.path.join(output_dir, 'poster_0_resized.jpg'))

print("이미지가 성공적으로 리사이즈되었습니다.")

이미지가 성공적으로 리사이즈되었습니다.


In [4]:
# augmentation

import cv2
import numpy as np
from PIL import Image, ImageEnhance
import os

def augment_image(image_path, output_dir):
    # 이미지 로드
    img = cv2.imread(image_path)

    # 회전 (Rotation)
    for angle in [0, 90, 180, 270]:
        rotated = cv2.rotate(img, angle)
        cv2.imwrite(os.path.join(output_dir, f'rotated_{angle}.jpg'), rotated)

    # 수평/수직 반전 (Flipping)
    flipped_h = cv2.flip(img, 1)  # 좌우 반전
    flipped_v = cv2.flip(img, 0)  # 상하 반전
    cv2.imwrite(os.path.join(output_dir, 'flipped_horizontal.jpg'), flipped_h)
    cv2.imwrite(os.path.join(output_dir, 'flipped_vertical.jpg'), flipped_v)

    # 크기 조정 및 자르기 (Zooming and Cropping)
    zoomed = cv2.resize(img, None, fx=1.2, fy=1.2)  # 20% 확대
    cv2.imwrite(os.path.join(output_dir, 'zoomed.jpg'), zoomed)

    # 이미지 자르기 (가운데 50% 크기로 자르기)
    h, w = img.shape[:2]
    cropped = img[int(h*0.25):int(h*0.75), int(w*0.25):int(w*0.75)]  # 가운데 50% 자르기
    cv2.imwrite(os.path.join(output_dir, 'cropped.jpg'), cropped)

    # 밝기 및 색상 변화 (Brightness and Color Shifts)
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # OpenCV에서 Pillow로 변환

    # 밝기 변화
    enhancer = ImageEnhance.Brightness(img_pil)
    for factor in [0.5, 1.5]:  # 50% 어둡게, 50% 밝게
        bright_img = enhancer.enhance(factor)
        bright_img.save(os.path.join(output_dir, f'bright_{factor}.jpg'))

    # 색상 변화
    enhancer = ImageEnhance.Color(img_pil)
    for factor in [0.5, 1.5]:  # 색상 감소 및 증가
        color_img = enhancer.enhance(factor)
        color_img.save(os.path.join(output_dir, f'color_{factor}.jpg'))

# 사용 예시
input_image_path = r'C:\ex\project_poster\preprocessed\poster_0_resized.jpg'
output_directory = r'C:\ex\project_poster\preprocessed'
os.makedirs(output_directory, exist_ok=True)

augment_image(input_image_path, output_directory)

print("데이터 증강이 완료되었습니다.")


데이터 증강이 완료되었습니다.


In [6]:
# 캐니 에지 검출

import cv2 as cv

img=cv.imread(r'C:\ex\project_poster\preprocessed\poster_0_resized.jpg')	# 영상 읽기

gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)

canny1=cv.Canny(gray,50,150)	# Tlow=50, Thigh=150으로 설정
canny2=cv.Canny(gray,100,200)	# Tlow=100, Thigh=200으로 설정

cv.imshow('Original',gray)
cv.imshow('Canny1',canny1)
cv.imshow('Canny2',canny2)

cv.waitKey()
cv.imwrite()
cv.destroyAllWindows()

In [12]:
# 경계선 찾기

import cv2 as cv
import numpy as np

img=cv.imread(r'C:\ex\project_poster\preprocessed\poster_0_resized.jpg')	 # 영상 읽기
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
canny=cv.Canny(gray,100,200) 

contour,hierarchy=cv.findContours(canny,cv.RETR_LIST,cv.CHAIN_APPROX_NONE)

lcontour=[]   
for i in range(len(contour)):
    if contour[i].shape[0]>100:	# 길이가 100보다 크면
        lcontour.append(contour[i])
    
cv.drawContours(img,lcontour,-1,(0,255,0),3)
             
cv.imshow('Original with contours',img)    
cv.imshow('Canny',canny)    

cv.waitKey()
cv2.imwrite(r'C:\ex\project_poster\preprocessed\counter.jpg', img)
cv2.imwrite(r'C:\ex\project_poster\preprocessed\canny.jpg', canny)
cv.destroyAllWindows()

In [13]:
# 슈퍼 화소 분할

import skimage
import numpy as np
import cv2 as cv

img=cv.imread(r'C:\ex\project_poster\preprocessed\poster_0_resized.jpg')
cv.imshow('poster image',cv.cvtColor(img,cv.COLOR_RGB2BGR))

slic1=skimage.segmentation.slic(img,compactness=20,n_segments=600)
sp_img1=skimage.segmentation.mark_boundaries(img,slic1)
sp_img1=np.uint8(sp_img1*255.0)

slic2=skimage.segmentation.slic(img,compactness=40,n_segments=600)
sp_img2=skimage.segmentation.mark_boundaries(img,slic2)
sp_img2=np.uint8(sp_img2*255.0)

cv.imshow('Super pixels (compact 20)',cv.cvtColor(sp_img1,cv.COLOR_RGB2BGR))
cv.imshow('Super pixels (compact 40)',cv.cvtColor(sp_img2,cv.COLOR_RGB2BGR))

cv.waitKey()
cv2.imwrite(r'C:\ex\project_poster\preprocessed\super_pixel_20.jpg', cv.cvtColor(sp_img1,cv.COLOR_RGB2BGR))
cv2.imwrite(r'C:\ex\project_poster\preprocessed\super_pixel_40.jpg', cv.cvtColor(sp_img2,cv.COLOR_RGB2BGR))
cv.destroyAllWindows()

In [16]:
#gray scale

import cv2
import numpy as np

img = cv2.imread(r'C:\ex\project_poster\preprocessed\poster_0_resized.jpg')

img2 = img.astype(np.uint16)                # dtype 변경 ---①
b,g,r = cv2.split(img2)                     # 채널 별로 분리 ---②
#b,g,r = img2[:,:,0], img2[:,:,1], img2[:,:,2]
gray1 = ((b + g + r)/3).astype(np.uint8)    # 평균 값 연산후 dtype 변경 ---③

gray2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # BGR을 그레이 스케일로 변경 ---④
cv2.imshow('original', img)
cv2.imshow('gray1', gray1)
cv2.imshow('gray2', gray2)

cv2.waitKey()
cv2.imwrite(r'C:\ex\project_poster\preprocessed\gray1.jpg', gray1)
cv2.imwrite(r'C:\ex\project_poster\preprocessed\gray2.jpg', gray2)
cv2.destroyAllWindows()